<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/13_starnodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt install golang-go

In [2]:
!go version

go version go1.10.4 linux/amd64


In [47]:
%%writefile serv.go
package main

import (
    "encoding/json"
    "fmt"
    "net"
)

func main() {
    ln, _ := net.Listen("tcp", "localhost:8500")
    defer ln.Close()
    conn, _ := ln.Accept()
    handle(conn)
}

func handle(conn net.Conn) {
    defer conn.Close()
    dec := json.NewDecoder(conn)
    var msg []string
    _ = dec.Decode(&msg)
    fmt.Println(msg)
}

Overwriting serv.go


In [46]:
%%writefile cli.go
package main

import (
    "encoding/json"
    "net"
)

func main() {
    conn, _ := net.Dial("tcp", "localhost:8500")
    defer conn.Close()
    enc := json.NewEncoder(conn)
    msg := []string{"hola", "mundo"}
    enc.Encode(&msg)
}


Overwriting cli.go


In [48]:
%%script bash
go run serv.go &
sleep 1
go run cli.go &
sleep 1

[hola mundo]


In [16]:
%%script bash

function cosita {
    for i in {1..10}; do
        echo "${1} $i"
        sleep 1
    done
}

cosita hola &
cosita chao &

hola 1
chao 1
hola 2
chao 2
hola 3
chao 3
hola 4
chao 4
hola 5
chao 5
hola 6
chao 6
chao 7
hola 7
chao 8
hola 8
hola 9
chao 9
hola 10
chao 10


In [89]:
%%writefile cmdlineargs.go
package main

import (
    "fmt"
    "os"
)

func main() {
    for i, arg := range os.Args {
        fmt.Println(i, arg)
    }
}

Overwriting cmdlineargs.go


In [90]:
!go run cmdlineargs.go hola mundo estos son \
                       parametros localhost:6000 "1 2 3"

0 /tmp/go-build004212734/b001/exe/cmdlineargs
1 hola
2 mundo
3 estos
4 son
5 parametros
6 localhost:6000
7 1 2 3


In [77]:
%%writefile nodo.go
package main

import (
    "encoding/json"
    "fmt"
    "net"
    "os"
)

type Msg struct {
    Command     string
    Hostname    string
    List        []string
}

var friends []string
var local   string
var end     chan bool

func serv() {
    fmt.Println("(", local, ")")
    ln, _ := net.Listen("tcp", local)
    defer ln.Close()
    for {
        conn, _ := ln.Accept()
        go handle(conn)
    }
}
func handle(conn net.Conn) {
    defer conn.Close()
    dec := json.NewDecoder(conn)
    var msg Msg
    if err := dec.Decode(&msg); err == nil {
        switch msg.Command {
        case "hello":
            resp := Msg{"hey", local, friends}
            enc := json.NewEncoder(conn)
            if err := enc.Encode(&resp); err == nil {
                for _, friend := range friends {
                    fmt.Println(local, "introduces", msg.Hostname,
                                "to", friend)
                    send(friend, "meet new friend", []string{msg.Hostname})
                }
            }
            friends = append(friends, msg.Hostname)
            fmt.Println(local, "updated list", friends)
        case "meet new friend":
            friends = append(friends, msg.List...)
            fmt.Println(local, "new friend", msg.List)
        case "finish":
            end<-true
        }
    }
}
func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, local, list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        fmt.Println(local, "sent", msg)
        if command == "hello" {
            dec := json.NewDecoder(conn)
            var resp Msg
            if err := dec.Decode(&resp); err == nil {
                friends = append(friends, resp.List...)
                fmt.Println(local, "recibí", resp.List)
            }
        }
    }
}
func main() {
    end = make(chan bool)
    local = os.Args[1]
    go serv()
    if len(os.Args) == 3 {
        remote := os.Args[2]
        friends = append(friends, os.Args[2])
        send(remote, "hello", []string{})
    }
    <-end
}

Overwriting nodo.go


In [78]:
!go build nodo.go

# command-line-arguments
./nodo.go:48:29: cannot use msg.List (type []string) as type string in append


In [53]:
%%writefile nodekiller.go
package main

import (
    "encoding/json"
    "fmt"
    "net"
    "os"
)

type Msg struct {
    Command string
    Hostname string
    List []string
}

func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, "", list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        fmt.Println("killing", remote)
    }
}

func main() {
    for _, remote := range os.Args[1:] {
        send(remote, "finish", []string{})
    }
}

Overwriting nodekiller.go


In [0]:
!go build nodekiller.go

In [56]:
%%script bash

./nodo localhost:8100 &
sleep 1
./nodekiller localhost:8100

( localhost:8100 )
killing localhost:8100


In [65]:
%%script bash
#kill -9 2607

ps ax

    PID TTY      STAT   TIME COMMAND
      1 ?        Ss     0:00 /bin/bash -e /datalab/run.sh
      8 ?        Sl     0:07 /tools/node/bin/node /datalab/web/app.js
     18 ?        Sl     0:13 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
    115 ?        Ss     0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
    123 ?        Ssl    0:14 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-f8e8c835-15a6-4a71-9466-774c5fa0be59.json
   3488 ?        S      0:00 bash
   3489 ?        R      0:00 ps ax


In [76]:
%%script bash

./nodo localhost:8000 &
sleep 2
./nodo localhost:8001 localhost:8000 &
sleep 2
./nodo localhost:8002 localhost:8000 &
sleep 2
./nodo localhost:8003 localhost:8002 &
sleep 2
./nodo localhost:8004 localhost:8001 &
sleep 2
./nodo localhost:8005 localhost:8003 &

sleep 2

./nodekiller localhost:8000 \
             localhost:8001 \
             localhost:8002 \
             localhost:8003 \
             localhost:8004 \
             localhost:8005


( localhost:8000 )
( localhost:8001 )
localhost:8001 sent {hello localhost:8001 []}
localhost:8000 updated list [localhost:8001]
localhost:8001 recibí []
( localhost:8002 )
localhost:8000 introduces localhost:8002 to localhost:8001
localhost:8000 sent {meet new friend localhost:8000 [localhost:8002]}
localhost:8000 updated list [localhost:8001 localhost:8002]
localhost:8002 sent {hello localhost:8002 []}
localhost:8001 new friend [localhost:8002]
localhost:8002 recibí [localhost:8001]
( localhost:8003 )
localhost:8003 sent {hello localhost:8003 []}
localhost:8002 introduces localhost:8003 to localhost:8000
localhost:8003 recibí [localhost:8000 localhost:8001]
localhost:8002 sent {meet new friend localhost:8002 [localhost:8003]}
localhost:8000 new friend [localhost:8003]
localhost:8002 introduces localhost:8003 to localhost:8001
localhost:8002 sent {meet new friend localhost:8002 [localhost:8003]}
localhost:8002 updated list [localhost:8000 localhost:8001 localhost:8003]
localhost:8001 